In [10]:
!pip install webdriver-manager

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import sys
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd

# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(3)

# Naver API key 입력
client_id = '' 
client_secret = ''

# selenium으로 검색 페이지 불러오기 #
naver_urls = []
postdate = []
titles = []

# 검색어 입력
keywords = input("검색할 키워드를 입력해주세요 (쉼표로 구분):")
keyword_list = keywords.split(',')
encText = urllib.parse.quote(' '.join(keyword_list))

# 검색을 끝낼 페이지 입력
end = input("\n크롤링을 끝낼 위치를 입력해주세요. (기본값:1, 최대값:100):")  
if end == "":
    end = 1
else:
    end = int(end)
print("\n 1 ~ ", end, "페이지 까지 크롤링을 진행 합니다")

# 한번에 가져올 페이지 입력
display = input("\n한번에 가져올 페이지 개수를 입력해주세요.(기본값:10, 최대값: 100):")
if display == "":
    display = 10
else:
    display = int(display)
print("\n한번에 가져올 페이지 : ", display, "페이지")

for start in range(1, end * display + 1, display):
    url = f"https://openapi.naver.com/v1/search/blog?query={encText}&start={start}&display={display}"  # JSON 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if rescode == 200:
        response_body = response.read()
        
        data = json.loads(response_body.decode('utf-8'))['items']
        for row in data:
            if 'blog.naver' in row['link']:
                naver_urls.append(row['link'])
                postdate.append(row['postdate'])
                title = row['title']
                # html태그제거
                pattern1 = '<[^>]*>'
                title = re.sub(pattern=pattern1, repl='', string=title)
                titles.append(title)
        time.sleep(2)
    else:
        print("Error Code:" + rescode)

###naver 기사 본문 및 제목 가져오기###

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

contents = []
comments_texts = []
try:
    for i in naver_urls:
        print(i)
        driver.get(i)
        time.sleep(5)  # 대기시간 변경 가능

        iframe = driver.find_element(By.ID , "mainFrame") # id가 mainFrame이라는 요소를 찾아내고 -> iframe임
        driver.switch_to.frame(iframe) # 이 iframe이 내가 찾고자하는 html을 포함하고 있는 내용

        source = driver.page_source
        html = BeautifulSoup(source, "html.parser")
        
        # 기사 텍스트만 가져오기
        content = html.select("div.se-main-container")
        #  list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')
        content = content.replace('\n', '')
        content = content.replace('\u200b', '')
        contents.append(content)

    news_df = pd.DataFrame({'title': titles, 'content': contents, 'date': postdate})
    news_df.to_csv('blog.csv', index=False, encoding='utf-8-sig')
except:
    contents.append('error')
    news_df = pd.DataFrame({'title': titles, 'content': contents, 'date': postdate})
    news_df.to_csv('blog.csv', index=False, encoding='utf-8-sig')

In [22]:
import pandas as pd 

df = pd.read_csv('blog500.csv')
df

,title,content,date
0,2024년 12월 성수 팝업 - tvN 홀리데이 파티 (팝업 기간은...,[안녕하세여! 여러분~! 여러분~ 2025년 되었습니다!! 새해 복 많이 받으세...,20250101
1,"[성수] 모푸샌드 팝업 스토어, 12월 마무리는 귀여운...","[ 예삐 학교가 12월 26, 27일을 재량 휴업일로 지정하는 바람에 휴가를 쓸 수...",20241227
2,12월 2주차 주간일기4 - 모푸샌드 mofusand 성수 팝업...,[불과 3년 전까지만 해도 우리 집은한 달에 한두 번은 쇼핑을 하러이마트나 코스트코...,20250102
3,유니클로 팝업 성수 히트텍 요원 되기?,[한때는 불매운동도 하더만 점점 이미지 좋아져서(?)성수 팝업도 오픈했어요하긴 겨울...,20241116
4,12월 성수 팝업 스토어 르쿠르제 팝업 예약없이 방문후기 (ft....,[ 12월 성수 팝업스토어 일정 중 제일 핫한 르쿠르제 팝업 &lt;Unfo...,20241206
...,...,...,...
495,"성수 팝업 핫플 추천, 빼빼로 미니 팝업 이벤트 참여해요!",[ 다양한 팝업스토어 구경하는 재미가 잇는성수동에서 이번에 빼빼로 팝업스토어가 오픈...,20241020
496,"뉴발란스 성수 993 팝업스토어 오픈, 아트갤러리 방문기",[본 포스팅은 ‘뉴발란스’ 로부터 소정의 원고료를 제공받아 솔직하게 작성된 후기입니...,20241017
497,퍼셀 긱 사이언티스트 랩 성수 팝업(1등 당첨 방문후기),[퍼셀을 알게 된건 신세계면세점!퍼셀 XXL 메이크오버 스틱 구매해서 써봤었는데코옆...,20241024
498,"11월 성수 팝업, 핫플 돌다가... 패딩 더현대서울팝업 정보 겟",[본 포스팅은 듀베티카로부터소정의 원고료를 제공받아 작성되었습니다 요즘 특히 핫한 ...,20241110


In [ ]:
# !pip install selenium
# !pip install beautifulsoup4
# !pip install tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from tqdm import tqdm
import json

# 웹 드라이버 선택
driver = webdriver.Chrome()

# 웹 드라이버 실행
driver.get('https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query=%EC%84%B1%EC%88%98+%ED%8C%9D%EC%97%85')


"""
셀레니움으로 사이트 주소 리스트를 저장하는 과정입니다.
get_site 함수는 사이트 각 설정 값 적용 및 게시물별 주소를 저장합니다.
"""
# 사이트 주소 담을 리스트
site_list = []

# 옵션버튼 최초 1회만 클릭하기 위한 변수
option_button = 0

def get_site(year, size):
    # 옵션버튼 최초 1회만 클릭하도록 설정
    global option_button
    if option_button == 0:
        driver.find_element(By.CSS_SELECTOR, 'i.spnew.ico_filter_arr').click()
        option_button += 1
        driver.implicitly_wait(3)

    # 기간 선택 화살표 클릭
    driver.find_element(By.CSS_SELECTOR, 'i.spnew.ico_check').click()
    driver.implicitly_wait(3)

    # 시작하는 날짜 선택
    driver.find_element(By.CSS_SELECTOR, 'a.spnew_bf.ico_calendar._start_trigger').click()
    driver.implicitly_wait(3)

    # year년 1월 1일 선택
    driver.find_element(By.XPATH, f'//div[@class="group_select _list_root"]//strong[contains(text(), "년(Year)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="{year}"]').click()
    driver.implicitly_wait(3)
    driver.find_element(By.XPATH, '//div[@class="group_select _list_root"]//strong[contains(text(), "월(Month)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="1"]').click()
    driver.implicitly_wait(3)
    driver.find_element(By.XPATH, '//div[@class="group_select _list_root"]//strong[contains(text(), "일(Day)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="1"]').click()
    driver.implicitly_wait(3)

    # 끝나는 날짜 선택
    driver.find_element(By.CSS_SELECTOR, 'a.spnew_bf.ico_calendar._end_trigger').click()
    driver.implicitly_wait(3)

    # year년 12월 31일 선택
    driver.find_element(By.XPATH, f'//div[@class="group_select _list_root"]//strong[contains(text(), "년(Year)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="{year}"]').click()
    driver.implicitly_wait(3)
    driver.find_element(By.XPATH, '//div[@class="group_select _list_root"]//strong[contains(text(), "월(Month)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="12"]').click()
    driver.implicitly_wait(3)
    driver.find_element(By.XPATH, '//div[@class="group_select _list_root"]//strong[contains(text(), "일(Day)")]/following-sibling::div[@class="select_cont"]//ul[@class="lst_item _ul"]/li[@data-value="31"]').click()
    driver.implicitly_wait(3)
    
    # 적용 버튼 클릭
    driver.find_element(By.CSS_SELECTOR, 'button.btn_apply._apply_btn').click()
    driver.implicitly_wait(3)

    # 1000개 이상이 될때까지 스크롤
    while len(driver.find_elements(By.CSS_SELECTOR, 'a.title_link')) < size:
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)
        driver.implicitly_wait(3)

    # 진행 상황 확인을 위해 글로벌 변수화(몰라도 됨)
    global href_list

    # 개별 사이트 주소를 리스트에 저장하고, 1000개 넘어갈 경우 제거
    href_list = [i.get_attribute('href') for i in driver.find_elements(By.CSS_SELECTOR, 'a.title_link')][:size]
    site_list.extend(href_list)

    # 날짜 설정 변경을 위해 가장 상단으로 스크롤 이동
    driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.HOME)
    driver.implicitly_wait(3)


"""
수집된 개별 사이트 주소를 이용해서 게시날짜, 제목, 본문을 저장하는 과정입니다.
개별 사이트는 iframe이라는 형식으로 되어 있습니다.
따라서 셀레니움이 해당사이트에 방문 후 iframe에 진입해야 크롤링이 가능합니다.
""" 
# 본문 담을 리스트
content_list = []

# 년도별 데이터 담을 딕셔너리
year_class = {}

# selenium으로 iframe에 들어가서 본문 긁어오기 함수
def get_content(url):
    # 사이트 방문
    driver.get(url)
    driver.implicitly_wait(3)

    # iframe영역으로 들어가기
    driver.switch_to.frame('mainFrame')
    driver.implicitly_wait(3)

    # 날짜
    dates = driver.find_element(By.CSS_SELECTOR, 'span.se_publishDate.pcol2').text.strip()
    driver.implicitly_wait(3)

    # 제목
    title = driver.find_element(By.CSS_SELECTOR, 'div.se-module.se-module-text.se-title-text').text.strip()
    driver.implicitly_wait(3)

    # 본문
    content = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'div.se-component.se-text.se-l-default')]
    content = ''.join(content).strip().replace('\n', ' ')
    driver.implicitly_wait(3)

    content_dict = {"date" : dates,
                    "title" :title,
                    "content" : content}

    # 년도만 출력
    year = content_dict['date'].split('.')[0]
    
    # {년도 : content_dict}틀 만들기 
    if year not in year_class:
        year_class[year] = []
    
    # 년도별 딕셔너리 형태로 저장
    year_class[year].append(content_dict)
    
    ## iframe 영역 나가기
    # driver.switch_to.default_content() 


"""
앞서 만든 get_site함수와 get_content함수를 통합한 함수입니다.
해당 함수에 
year_list = 수집할 연도 리스트
year_size = 연도별 글 개수
이 두가지만 입력하면 자동으로 돌아갑니다.
"""

# 연도별, 주소 수집 함수
def get_json(year_list, year_size):
    for i in tqdm(year_list):
        get_site(i, year_size)
        print(f'{i}년 사이트 주소 : {len(href_list)}개 수집')
        print('-'*50)
        print()

    print(f'총 사이트 주소 개수 : {len(site_list)}개')
    print('본문 수집 시작')

    # 본문 수집
    for site in tqdm(site_list):
        get_content(site)

    content_list.append(year_class)

    # 컨텐츠 목록 json파일로 만들기
    with open ('naver_blog.json', 'w', encoding='utf-8') as f:
        json.dump(content_list, f, ensure_ascii=False, indent=4) 


"""
사용할 때 변경할 부분은 이 아래가 전부입니다.
year_list와 year_size만 설정하세요.
"""
# 수집할 연도 리스트(2020~2025년)
year_list = [i for i in range(2020, 2026)]

# 수집할 연도별 글 개수
year_size = 1000

# 통합 함수 실행
get_json(year_list, year_size)


driver.close() # 해당 브라우저만 닫기
# driver.quit()  # 전체 브라우저 모두 닫기